PID Controller Notebook
* Ziegler-Nichols Method

Tutors:
* OpenAI's ChatGPT
* Anthropic's AI Claude

2nd order system script

In [ ]:
"""
Abstract:
This script models a second-order mass-spring-damper system and tunes a PID controller using the Ziegler-Nichols method.
The script automatically searches for the ultimate gain (Ku) and the ultimate period (Tu) by increasing Kp until sustained
oscillations are detected. After tuning the PID parameters based on the Ziegler-Nichols rules, the script simulates the
system using these PID gains and calculates key performance metrics, such as steady-state error, rise time, settling time,
and overshoot. Manual fine-tuning of the PID parameters is also offered. The system dynamics are simulated using the
solve_ivp solver for accurate integration of the second-order dynamics.
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.signal import find_peaks

# System and simulation parameters
m = 1.0  # Mass of the system
b = 5.0  # Damping coefficient
k = 20.0  # Spring constant
setpoint = 1.0  # Desired position (target or reference value)

# PID controller class to implement the proportional, integral, and derivative terms
class PIDController:
    def __init__(self, Kp, Ki, Kd):
        self.Kp = Kp  # Proportional gain
        self.Ki = Ki  # Integral gain
        self.Kd = Kd  # Derivative gain
        self.previous_error = 0  # To store the error from the previous time step (for derivative calculation)
        self.integral = 0  # To accumulate the integral of the error over time
        self.previous_time = 0  # To store the previous time step (for time difference calculation)

    # Control function to calculate the PID control signal based on error and time
    def control(self, t, error):
        dt = t - self.previous_time  # Time difference since the last control update
        if dt > 0:
            self.integral += error * dt  # Accumulate the integral of the error
            derivative = (error - self.previous_error) / dt  # Calculate the derivative of the error
        else:
            derivative = 0
        # PID control output: sum of proportional, integral, and derivative terms
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        # Store the current error and time for the next step
        self.previous_error = error
        self.previous_time = t
        return output

# System dynamics: defines the equations of motion for the mass-spring-damper system
def system_dynamics(t, state, controller):
    x, v = state  # x is position, v is velocity
    error = setpoint - x  # Error is the difference between the setpoint and the current position
    u = controller.control(t, error)  # Control signal calculated by the PID controller
    dxdt = v  # The rate of change of position is the velocity
    dvdt = (u - b * v - k * x) / m  # The rate of change of velocity is given by Newton's second law
    return [dxdt, dvdt]  # Return the rates of change for position and velocity

# Simulate system with the given PID controller
def simulate_system(controller, t_span, t_eval):
    sol = solve_ivp(
        fun=lambda t, y: system_dynamics(t, y, controller),  # System dynamics function
        t_span=t_span,  # The time range for the simulation
        y0=[0, 0],  # Initial condition: [position = 0, velocity = 0]
        t_eval=t_eval,  # Time points at which to store the solution
        method='RK45'  # Runge-Kutta integration method for accurate ODE solution
    )
    return sol.t, sol.y.T  # Return the time points and the system state (position, velocity)

# Detect sustained oscillations by analyzing peaks in the position signal
def detect_oscillations(y, t, num_peaks=4, tolerance=0.15, amplitude_threshold=0.1):
    peaks, _ = find_peaks(y, height=amplitude_threshold)  # Detect peaks in the position signal
    if len(peaks) < num_peaks:  # If fewer than the required number of peaks are detected, oscillations are not sustained
        return False, None

    peak_heights = y[peaks]  # Heights of the detected peaks
    peak_times = t[peaks]  # Times at which peaks occur

    # Check if the heights of the last few peaks are similar (within a tolerance)
    if np.std(peak_heights[-num_peaks:]) / np.mean(peak_heights[-num_peaks:]) > tolerance:
        return False, None

    # Check if the time intervals between the last few peaks are similar (within a tolerance)
    intervals = np.diff(peak_times[-num_peaks:])
    if np.std(intervals) / np.mean(intervals) > tolerance:
        return False, None

    # If both conditions are satisfied, return True and the estimated period of oscillation (2 times the mean interval)
    return True, np.mean(intervals) * 2

# Find the ultimate gain (Ku) and the ultimate period (Tu) using the Ziegler-Nichols method
def find_ultimate_gain(max_iterations=200, Kp_start=1.0, Kp_max=10000):
    Kp = Kp_start  # Start with an initial proportional gain
    t_span = (0, 100)  # Simulation time range
    t_eval = np.linspace(*t_span, 10000)  # Time points for evaluation

    for i in range(max_iterations):
        print(f"Iteration {i+1}/{max_iterations}, Kp = {Kp:.4f}")
        controller = PIDController(Kp, 0, 0)  # Use a pure P-controller for the Ziegler-Nichols method
        t, y = simulate_system(controller, t_span, t_eval)  # Simulate the system with the current Kp

        oscillating, Tu = detect_oscillations(y[:, 0], t)  # Detect oscillations and estimate the period
        if oscillating:  # If oscillations are detected, return the ultimate gain and period
            return Kp, Tu

        if Kp > Kp_max:  # If the proportional gain exceeds the maximum limit, stop the search
            print(f"Kp exceeded maximum value of {Kp_max}. Stopping search.")
            return None, None

        Kp *= 1.5  # Increase the proportional gain for the next iteration

    print(f"Couldn't find ultimate gain within {max_iterations} iterations.")
    return None, None

# Calculate performance metrics such as steady-state error, rise time, settling time, and overshoot
def calculate_metrics(t, y):
    steady_state_value = y[-1, 0]  # Final position value (steady-state value)
    steady_state_error = abs(setpoint - steady_state_value)  # Error at steady state

    # Calculate rise time: time taken to reach 63.2% of the setpoint (time constant)
    rise_time_index = np.argmax(y[:, 0] >= 0.632 * setpoint)
    rise_time = t[rise_time_index] if rise_time_index > 0 else None

    # Calculate settling time: time taken for the position to stay within 5% of the setpoint
    settling_time_index = np.argmax(np.all(np.abs(y[rise_time_index:, 0] - setpoint) <= 0.05 * setpoint))
    settling_time = t[settling_time_index + rise_time_index] if settling_time_index > 0 else None

    # Calculate overshoot: percentage of how much the position exceeds the setpoint
    overshoot = max(0, (np.max(y[:, 0]) - setpoint) / setpoint * 100)

    return {
        "Steady State Value": steady_state_value,
        "Steady State Error": steady_state_error,
        "Rise Time (63.2%)": rise_time,
        "Settling Time (5%)": settling_time,
        "Overshoot (%)": overshoot
    }

# Main simulation and control flow
print("Finding ultimate gain and period...")
Ku, Tu = find_ultimate_gain()  # Find the ultimate gain (Ku) and the ultimate period (Tu)

if Ku is None or Tu is None:
    print("Failed to find valid Ku and Tu. Please adjust system parameters and try again.")
else:
    print(f"Ultimate gain (Ku): {Ku:.4f}")
    print(f"Ultimate period (Tu): {Tu:.4f}")

    # Calculate initial PID parameters based on the Ziegler-Nichols method
    Kp = 0.6 * Ku
    Ti = Tu / 2
    Td = Tu / 8
    Ki = Kp / Ti * 1.5  # Increased integral action
    Kd = Kp * Td * 1.2  # Slightly increased derivative action

    print(f"Initial PID parameters: Kp = {Kp:.4f}, Ki = {Ki:.4f}, Kd = {Kd:.4f}")

    # Option for manual fine-tuning of the PID parameters


First order system script

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint
from scipy.signal import find_peaks

# Function to model a first-order system (e.g., a thermal system)
def system_model(y, t, u, a, b):
    dydt = (-y + b * u) / a  # First-order system dynamics
    return dydt

# PID Controller class
class PIDController:
    def __init__(self, K_p, K_i, K_d):
        self.K_p = K_p
        self.K_i = K_i
        self.K_d = K_d
        self.prev_error = 0
        self.integral = 0

    # Calculate the PID output
    def update(self, error, dt):
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt
        self.prev_error = error
        return self.K_p * error + self.K_i * self.integral + self.K_d * derivative

# System simulation with PID control
def simulate_system(K_p, K_i, K_d, t, y0, a, b):
    pid = PIDController(K_p, K_i, K_d)
    u = 0  # Control signal
    y = np.zeros_like(t)  # Output array
    y[0] = y0
    set_point = 1.0  # Desired setpoint (reference value)

    for i in range(1, len(t)):
        dt = t[i] - t[i-1]
        error = set_point - y[i-1]
        u = pid.update(error, dt)  # PID control signal
        dydt = system_model(y[i-1], t[i-1], u, a, b)  # Simplified model update
        y[i] = y[i-1] + dydt * dt  # Euler integration for system state update

    return y

# Automatically find the ultimate gain and period using the closed-loop method
def find_ultimate_gain_period(t, y):
    peaks, _ = find_peaks(y, height=0.002, distance=3)  # More sensitive peak detection
    peak_times = t[peaks]
    if len(peak_times) < 2:
        return None, None
    T_u = np.mean(np.diff(peak_times))  # Average time between peaks
    K_u = 1.0  # Manually set ultimate gain, adjust as needed
    return K_u, T_u

# Define simulation parameters
t = np.linspace(0, 100, 10000)  # Further increased time array and resolution
y0 = 0.5  # Changed initial condition
a, b = 1.5, 1.0  # System parameters

# Automatically tune K_p
def auto_tune_K_p(K_p_initial, a, b, max_K_p=100.0, increment=0.05):
    K_p = K_p_initial
    while K_p <= max_K_p:
        y_initial = simulate_system(K_p, 0, 0, t, y0, a, b)
        K_u, T_u = find_ultimate_gain_period(t, y_initial)
        if K_u is not None and T_u is not None:
            return K_p, y_initial, K_u, T_u
        K_p += increment  # Smaller increments for finer control
    return None, None, None, None

# Initial K_p value (can be adjusted automatically)
K_p_initial = 1.0
K_p_initial, y_initial, K_u, T_u = auto_tune_K_p(K_p_initial, a, b)

# Exit if tuning fails
if K_p_initial is None:
    print("Failed to find an oscillatory system response. Tuning aborted.")
else:
    # Apply Ziegler-Nichols Tuning Rules for PID
    K_p_zn = 0.6 * K_u
    K_i_zn = 2 * K_p_zn / T_u
    K_d_zn = K_p_zn * T_u / 8

    # Simulate the system with the tuned PID values
    y_tuned = simulate_system(K_p_zn, K_i_zn, K_d_zn, t, y0, a, b)

    # Output the table of PID values
    pid_table = pd.DataFrame({
        'Parameter': ['K_p', 'K_i', 'K_d'],
        'Value': [K_p_zn, K_i_zn, K_d_zn]
    })
    print("Tuned PID Parameters (Ziegler-Nichols Method):")
    print(pid_table.to_string(index=False))

    # Plot the system response before and after tuning
    plt.figure(figsize=(10, 6))
    plt.plot(t, y_initial, label=f'Before Tuning (K_p = {K_p_initial:.2f}, K_i = 0, K_d = 0)', linestyle='--')
    plt.plot(t, y_tuned, label=f'After Tuning (K_p = {K_p_zn:.2f}, K_i = {K_i_zn:.2f}, K_d = {K_d_zn:.2f})', linewidth=2)
    plt.axhline(1.0, color='gray', linestyle=':', label='Set Point')
    plt.title('System Response Before and After PID Tuning (Ziegler-Nichols Method)')
    plt.xlabel('Time')
    plt.ylabel('System Output')
    plt.legend()
    plt.grid(True)
    plt.show()


To ensure that the equations display correctly in a Google Colab **text box** (as opposed to a markdown cell), you'll need to avoid using LaTeX formatting like `$$`. Instead, you can write the equations in plain text using symbols for division and multiplication.


---

Let's compare the two scripts based on their structure, PID tuning methods, and system dynamics.

### 1. **System Dynamics**:
   - **The first-order system script** models a first-order system with the differential equation:  
     `dy/dt = (-y + b * u) / a`  
     which is simple and represents a thermal or similar system.

   - **The second-order system script** models a second-order system (mass-spring-damper) with the dynamics:  
     `dx/dt = v,`  
     `dv/dt = (u - b * v - k * x) / m`  
     where `x` is position, `v` is velocity, `b` is the damping coefficient, and `k` is the spring constant.

   **Key difference**: The second-order system is inherently more prone to oscillations (due to the mass-spring relationship), while the first-order system requires specific gains to induce oscillatory behavior.

### 2. **PID Controller Implementation**:
   - Both scripts implement a PID controller in a similar manner. However:
     - **The first-order system script** calculates PID output directly using proportional, integral, and derivative terms, which are updated in each iteration of the Euler integration loop.
     - **The second-order system script** also has a PID controller but uses the `control` function, which is passed to the system's ODE and is called by `solve_ivp` as part of the integration.

   **Key difference**: The second-order system script uses the `solve_ivp` solver (Runge-Kutta method), which can offer better accuracy for simulating second-order systems, while the first-order system script uses Euler integration, which is more suitable for simple first-order systems.

### 3. **Simulation and Solver**:
   - **The first-order system script** uses a manual loop with Euler integration to simulate the system.
   - **The second-order system script** leverages `solve_ivp`, which is more sophisticated and handles complex ODEs better than Euler integration.

   **Key difference**: The second-order system script may handle dynamic systems with greater accuracy, especially for systems prone to oscillations.

### 4. **Peak Detection and Oscillation Detection**:
   - **The first-order system script** uses `find_peaks` from SciPy to detect oscillations, but the sensitivity is set to detect very small peaks, which might not be ideal for all systems.
   - **The second-order system script** also uses `find_peaks` but with modified detection parameters for a less sensitive, more robust detection process. It requires more peaks to determine oscillations and also checks that the oscillations are sustained (i.e., not decaying too quickly).

   **Key difference**: The second-order system script includes more advanced logic for ensuring oscillations are sustained over time and checks the regularity of peak intervals and heights.

### 5. **Ultimate Gain Tuning**:
   - **The first-order system script** implements a Ziegler-Nichols tuning method by incrementing `Kp` until oscillations are detected. It assumes `Ku = 1`, which may not always hold true for all systems.
   - **The second-order system script** also applies a Ziegler-Nichols method but increases `Kp` more aggressively (multiplies by 1.5 each time) and starts at higher values. Additionally, it adjusts the PID gains with slight modifications for more aggressive tuning.

   **Key difference**: The second-order system script has a more aggressive search for the ultimate gain and adjusts the Ziegler-Nichols tuning parameters to account for practical considerations (e.g., increased integral and derivative terms).

### 6. **Performance Metrics**:
   - **The first-order system script** does not calculate performance metrics such as rise time, settling time, and overshoot.
   - **The second-order system script** calculates these performance metrics and provides a detailed summary, helping assess how well the PID controller performs.

   **Key difference**: The second-order system script provides more insight into the performance of the PID controller by calculating key performance metrics.

### 7. **Manual Fine-Tuning**:
   - **The first-order system script** relies purely on the Ziegler-Nichols method.
   - **The second-order system script** offers an option for manual fine-tuning of the PID parameters after the initial automatic tuning, allowing for human adjustments based on system behavior.

   **Key difference**: The second-order system script offers a more flexible approach by allowing manual fine-tuning.

### 8. **System Type**:
   - **The first-order system script** works on a first-order system (typically simpler, fewer oscillations unless gains are very specific).
   - **The second-order system script** works on a second-order system (mass-spring-damper), which inherently has the potential for oscillations due to its physical properties.

### Summary of Differences:
- **System Type**: First-order (first-order system script) vs. second-order (second-order system script).
- **Solver**: Euler integration (first-order system script) vs. `solve_ivp` (second-order system script).
- **Oscillation Detection**: Basic peak detection (first-order system script) vs. sustained oscillation detection (second-order system script).
- **Ultimate Gain Tuning**: Basic search for `Kp` (first-order system script) vs. more aggressive search with modified Ziegler-Nichols parameters (second-order system script).
- **Performance Metrics**: Not calculated (first-order system script) vs. detailed metrics (second-order system script).
- **Manual Fine-Tuning**: Not available (first-order system script) vs. available in the second-order system script.

---
